# 🧠 CerviScan-Net: Full Research Framework (2026)This notebook performs both binary and multi-class cervical cell classification using CerviScan-Net and baseline models.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np, os, matplotlib.pyplot as plt, seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
print('TensorFlow version:', tf.__version__)
print('GPUs available:', tf.config.list_physical_devices('GPU'))
DATA_PATH = './datasets/Herlev/'
BATCH_SIZE = 16
IMG_SIZE = (224,224)
EPOCHS = 30

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(DATA_PATH, image_size=IMG_SIZE, batch_size=BATCH_SIZE, validation_split=0.2, subset='training', seed=42)
val_ds = tf.keras.utils.image_dataset_from_directory(DATA_PATH, image_size=IMG_SIZE, batch_size=BATCH_SIZE, validation_split=0.2, subset='validation', seed=42)
class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print('Detected classes:', class_names)

In [ ]:
def build_cerviscan_net(input_shape=(224,224,3), num_classes=2):
    inp = keras.Input(shape=input_shape)
    swin = keras.applications.EfficientNetB0(include_top=False, input_shape=input_shape, weights='imagenet')
    eff = keras.applications.EfficientNetV2B0(include_top=False, input_shape=input_shape, weights='imagenet')
    swin_feat, eff_feat = swin(inp), eff(inp)
    x = keras.layers.Concatenate()([swin_feat, eff_feat])
    x = keras.layers.Conv2D(128, 1, activation='relu')(x)
    x = keras.layers.Reshape((-1, 128))(x)
    x = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(x)
    x = keras.layers.Attention()([x, x])
    x = keras.layers.GlobalAveragePooling1D()(x)
    out = keras.layers.Dense(num_classes, activation='softmax')(x)
    return keras.Model(inp, out, name='CerviScanNet')

In [ ]:
model = build_cerviscan_net(num_classes=NUM_CLASSES)
model.compile(optimizer=keras.optimizers.Adam(1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

In [ ]:
preds = np.argmax(model.predict(val_ds), axis=1)
true = np.concatenate([y for x, y in val_ds], axis=0)
print(classification_report(true, preds, target_names=class_names))